In [ ]:
%load_ext autoreload
%autoreload 2
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))

In [ ]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
from pandas import Timedelta
# from optiml.utils import sf
import time
from optiml.utils.sf import logger, sql_to_df, run_sql, conn, session
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime


In [ ]:
# Q1: if Cluster 1 and 3 are active, 2 is idle, and num_clusters is set to 2, does #2 shutoff?
#    1b) are credits saved?
# Q2: if cluster 1 and 2 are active, and 2 is shutoff before 60 seconds of uptime, are we billed
#     at minimum of 60 secs like base warehouse cluster? or charged for usage?
#     - yes, minimum of 60 secs billed

# algos:
# 1) if warehouse is idle, shut down warehouse
# 2) estimate total number of active clusters
#   a) use total number of running queries
#   b) use total pct of provisioned 
# set max_clusters to that number of active clusters
# 3) if multicluster type is STANDARD and warehouse is idle set to ECONOMY
#    if not idle and total active clusters > 1 (or total active clusters = max clusters)
#        then set to ECONOMY

In [ ]:
# accidental runs of the same scenario (no warehouse mgmt; standard scaling policy); shows unexpected variations!
wh_names =  ['test_control_a67fc85c_9de9_400e_8e49_6c93dfcef9db',
 'test_suspend_idle_after_one_minute_63fcffb5_6a18_4e6c_bb59_824a2a1862b2',
 'test_minimize_clusters_if_no_queuing_ec3ecce1_6661_4d0d_a93c_f5b678be9a19',
 'test_economy_53142176_000d_4587_8911_4a6115c4ebac',
 'test_economy_up_standard_down_e9316185_aea0_4325_ba8c_775c3ad551ed']

path = 'test_control_a67fc85c_9de9_400e_8e49_6c93dfcef9db_2023_0614_013902.pkl'


In [ ]:
df = pd.read_pickle(path)

In [ ]:
# # wh_name='DEMO'
# for wh_name in wh_names:
#     print(wh_name.upper())
#     import plotly.express as px
#     fig = px.line(df[df.name == wh_name.upper()], x='ts', y='started_clusters')
#     fig.show()
#     fig = px.line(df[df.name == wh_name.upper()], x='ts', y=['running', 'queued'])
#     fig.show()
#     fig = px.line(df[df.name == wh_name.upper()], x='ts', y=['available', 'provisioning', 'quiescing'])
#     fig.update_yaxes(autorange="reversed")

#     fig.show()

In [ ]:
run_sql("set lookback_days=2")
wh_status = f"""with qh as (
select 
    *,
    md5(warehouse_name) as warehouse_id,
      warehouse_name || ':' || cluster_number || ':' || query_id as warehouse_query,
    TIMESTAMPADD( millisecond , -execution_time,end_time) as execution_start_time
from table(information_schema.query_history_by_user('saf', result_limit=>10000))
where execution_status != 'RUNNING'
and warehouse_size is not null
and start_time > dateadd('days',-$lookback_days, current_timestamp())
order by start_time desc
),
seconds as (
    select
        dateadd(
            'second',
            '-' || row_number() over (order by seq4() asc),
            dateadd('day', '+1', date_trunc('second', current_timestamp))
        ) as second_start,
        dateadd('second', '+1', second_start) as timestamp
    from table(generator(rowcount => (3600*24 * 100)))
    qualify second_start between (select min(start_time) from qh) and (select max(end_time) from qh)
),
warehouse_cluster_seconds as (
    select * from 
    seconds
    cross join 
    (select distinct md5(warehouse_name) as warehouse_id, warehouse_name, cluster_number from qh) w

)
-- select * from warehouse_seconds;
select
    seconds.timestamp,
    seconds.warehouse_name,
    seconds.warehouse_id,
    seconds.cluster_number,
    count(queries.query_id) as num_queries
from warehouse_cluster_seconds seconds
left join qh as queries
    -- on date_trunc('hour', seconds.timestamp)=queries.query_hour -- NEW: equi-join condition
    on seconds.warehouse_id = queries.warehouse_id
    and seconds.cluster_number = queries.cluster_number
    and seconds.timestamp -- range join condition
      between date_trunc('second', queries.execution_start_time) and date_trunc('second', queries.end_time)
group by 1,2,3,4
"""

wdf = sql_to_df(wh_status)
display(wdf)


events = f"""
select *
from snowflake.account_usage.warehouse_events_history
where timestamp > dateadd('days',-$lookback_days, current_timestamp())
"""

whedf = sql_to_df(events)
display(whedf)

statuses = sql_to_df("""
with a as (
select 
    *,
    md5(warehouse_name) as warehouse_id,
      warehouse_name || ':' || cluster_number || ':' || query_id as warehouse_query,
    TIMESTAMPADD( millisecond , -execution_time,end_time) as execution_start_time
from table(information_schema.query_history_by_user('saf', result_limit=>10000))
where execution_status != 'RUNNING'
and warehouse_size is not null
and start_time > dateadd('days',-$lookback_days, current_timestamp())
)
select
    warehouse_name,
    execution_status,
    count(*)
from a 
    group by 1,2
""")
                

In [ ]:
from plotly.subplots import make_subplots

# qdf['warehouse_query'] = qdf.warehouse_name + " : " + qdf.query_id

wdf['timestamp_est'] = wdf.timestamp + pd.Timedelta(hours=3)
whedf['timestamp_est'] = whedf.timestamp + pd.Timedelta(hours=3)


import plotly.express as px
# fig = px.line(df[df.name == wh_name.upper()], x='ts', y='started_clusters')
# fig.show()
# fig = px.line(df[df.name == wh_name.upper()], x='ts', y=['running', 'queued'])
# fig.show()
# fig = px.line(df[df.name == wh_name.upper()], x='ts', y=['available', 'provisioning', 'quiescing'])
# fig.update_yaxes(autorange="reversed")

# fig.show()

# wh_name = wh_names[0]

wh_name_subset = wh_names # wh_names[0:1]
figures = []
for wh_name in wh_name_subset:
    print(wh_name)
    wh_figures = [
        px.line(df[df.name == wh_name.upper()], x='ts', y='started_clusters', title=wh_name.upper()),
        px.line(df[df.name == wh_name.upper()], x='ts', y=['running', 'queued']),
        px.line(df[df.name == wh_name.upper()], x='ts', y=['available', 'provisioning', 'quiescing']),
        # px.scatter(whedf[whedf.warehouse_name == wh_name.upper()].sort_values('warehouse_name'), x="timestamp_est", y="event_name", color='event_name', hover_data=['cluster_number', 'event_name', 'event_reason']),
        px.area(wdf[wdf.warehouse_name == wh_name.upper()].sort_values(['timestamp_est', 'cluster_number']), x='timestamp_est', y=['num_queries'], color='cluster_number'),
    ]
    figures = wh_figures
    # figures.extend(wh_figures)
    
    fig = make_subplots(rows=len(figures), cols=1, shared_xaxes=True, vertical_spacing=0.02) 

    for i, figure in enumerate(figures):
        print(i)
        for trace in range(len(figure["data"])):
            fig.append_trace(figure["data"][trace], row=i+1, col=1)
            fig.update_xaxes(range=[df.ts.min() - pd.Timedelta(seconds=20), df.ts.max()], row=i+1, col=1)



    fig.update_xaxes(showgrid=True,minor=dict(showgrid=True))
    fig.update_yaxes(showgrid=True,minor=dict(showgrid=True))

    # fig.update_layout(height=len(wh_name_subset)*1000)
    fig.update_layout(height=1000)
    fig.update_xaxes(type='date')
    fig.update_yaxes(row=3, col=1, autorange='reversed')


    fig.show()
print("done")
# next: try just one layer of test query

In [ ]:
spend_sql = """
    select * 
from table(information_schema.WAREHOUSE_METERING_HISTORY( 
    date_range_start => dateadd('days',-2,current_date())   
    )
)
"""

spend_df = sql_to_df(spend_sql)
# display(spend_df)

In [ ]:
spend_df[spend_df.warehouse_name.str.lower().isin(wh_names)]

In [ ]:
import plotly.express as px
fig = px.bar(spend_df[spend_df.warehouse_name.str.lower().isin(wh_names)], x = 'start_time', y = 'credits_used', color='warehouse_name', barmode='group')
fig.show()

In [ ]:
statuses = """
with a as (
select 
    *,
    md5(warehouse_name) as warehouse_id,
      warehouse_name || ':' || cluster_number || ':' || query_id as warehouse_query,
    TIMESTAMPADD( millisecond , -execution_time,end_time) as execution_start_time
from table(information_schema.query_history_by_user('saf', result_limit=>10000))
where execution_status != 'RUNNING'
and warehouse_size is not null
and start_time > dateadd('days',-$lookback_days, current_timestamp())
)
select
    warehouse_name,
    execution_status,
    count(*) num_queries
from a 
    group by 1,2;
    """
statuses_df = sql_to_df(statuses)
import plotly.express as px
fig = px.bar(statuses_df[statuses_df.warehouse_name.str.lower().isin(wh_names)], x = 'warehouse_name', y = 'num_queries', color='execution_status', barmode='group')
fig.show()